In [2]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO

# 경로 설정
TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
FILTERED_DIR = "D:/데이콘 250519 대회/filtered_train"

# YOLOv8 모델 로드 (가벼운 yolov8n.pt 사용)
model = YOLO('yolov8n.pt')

def is_full_vehicle(detections, img_shape):
    h, w = img_shape[:2]

    for det in detections:
        cls = int(det.cls)
        # 차량 클래스만 필터링 (COCO 기준: 2=car, 5=bus, 7=truck)
        if cls in [2, 5, 7]:
            x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
            box_w = x2 - x1
            box_h = y2 - y1
            area = box_w * box_h
            img_area = h * w
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            aspect_ratio = box_w / box_h

            # 1) 차량이 이미지 중앙 근처에 위치 (너무 치우치면 제외)
            if not (0.35*w < center_x < 0.65*w and 0.35*h < center_y < 0.65*h):
                continue

            # 2) 차량 박스가 이미지 대비 충분히 큰 면적 (예: 35% 이상)
            if area / img_area < 0.35:
                continue

            # 3) 차량 박스 비율 필터링
            if aspect_ratio < 0.8 or aspect_ratio > 2.5:
                continue

            # 조건 모두 만족하는 차량이 있으면 True 반환
            return True

    return False

# 필터링 수행
os.makedirs(FILTERED_DIR, exist_ok=True)
class_dirs = sorted(os.listdir(TRAIN_DIR))

filtered_count = 0
total_count = 0

for cls in tqdm(class_dirs, desc="클래스별 필터링"):
    input_dir = os.path.join(TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)

    image_paths = glob(os.path.join(input_dir, "*.jpg"))
    for img_path in image_paths:
        total_count += 1
        img = cv2.imread(img_path)
        results = model(img, verbose=False)[0]

        if is_full_vehicle(results.boxes, img.shape):
            filename = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_dir, filename), img)
            filtered_count += 1

print(f"전체 이미지 수: {total_count}")
print(f"필터링된 차량 외관 이미지 수: {filtered_count}")
print(f"정제 비율: {filtered_count / total_count:.2%}")


클래스별 필터링: 100%|██████████| 396/396 [34:01<00:00,  5.16s/it]

전체 이미지 수: 33137
필터링된 차량 외관 이미지 수: 31822
정제 비율: 96.03%


In [2]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

import cv2
from ultralytics import YOLO

# -----------------------
# GPU 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"[INFO] {len(gpus)} GPU(s) available: {[gpu.name for gpu in gpus]}")
else:
    print("[INFO] No GPU available. Training will use CPU.")

# -----------------------
# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# -----------------------
# 경로 설정
ORIGINAL_TRAIN_DIR = "D:/데이콘 250519 대회/open/train"        # 원본 원천 데이터셋
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/filtered_train"    # 필터링 결과 저장 폴더
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# -----------------------
# YOLOv8 차량 외관 필터링 함수

VEHICLE_CLASSES = [2, 5, 7]  # car, bus, truck 클래스 번호

yolo_model = YOLO('yolov8n.pt')  # 필요시 yolov8s.pt 등으로 변경 가능

def is_full_vehicle(detections, img):
    h, w = img.shape[:2]
    img_area = h * w

    for det in detections:
        cls = int(det.cls)
        if cls not in VEHICLE_CLASSES:
            continue

        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        box_w, box_h = x2 - x1, y2 - y1
        area = box_w * box_h
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        aspect_ratio = box_w / box_h

        # 차량 중앙 근처 위치 조건
        if not (0.35 * w < center_x < 0.65 * w and 0.35 * h < center_y < 0.65 * h):
            continue

        # 차량 박스 크기 최소 비율 조건
        if area / img_area < 0.35:
            continue

        # 차량 비율 조건 (트렁크/보닛 열림 등 제거용)
        if aspect_ratio < 0.8 or aspect_ratio > 2.5:
            continue

        # 위 조건 모두 통과 시 전체 차량 외관 포함으로 판단
        return True

    return False

# -----------------------
# 차량 외관 필터링 수행

print("차량 외관 이상치 필터링 시작...")
os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)
class_dirs = sorted(os.listdir(ORIGINAL_TRAIN_DIR))

filtered_count = 0
total_count = 0

for cls in tqdm(class_dirs, desc="클래스별 필터링"):
    input_dir = os.path.join(ORIGINAL_TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_TRAIN_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)

    image_paths = glob(os.path.join(input_dir, "*.jpg"))
    for img_path in image_paths:
        total_count += 1
        img = cv2.imread(img_path)
        results = yolo_model(img, verbose=False)[0]

        if is_full_vehicle(results.boxes, img):
            filename = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_dir, filename), img)
            filtered_count += 1

print(f"🔍 전체 이미지 수: {total_count}")
print(f"✅ 필터링된 차량 외관 이미지 수: {filtered_count}")
print(f"🧼 이상치 제거 완료. 정제 비율: {filtered_count / total_count:.2%}")

# -----------------------
# 학습 데이터 준비

label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# -----------------------
# 데이터 전처리 함수

def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# -----------------------
# 모델 생성

def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    x = layers.Dense(num_classes, activation='softmax')(base.output)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# -----------------------
# Stratified K-Fold 학습

skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1} 시작 ###")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CFG['EPOCHS'],
        verbose=1
    )

# -----------------------
# 테스트 데이터 추론 및 제출 파일 생성

test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

print("테스트 데이터 추론 시작...")
preds = model.predict(test_ds)
print("추론 완료")

submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")


[INFO] 1 GPU(s) available: ['/physical_device:GPU:0']
차량 외관 이상치 필터링 시작...


클래스별 필터링: 100%|██████████| 396/396 [36:02<00:00,  5.46s/it]


🔍 전체 이미지 수: 33137
✅ 필터링된 차량 외관 이미지 수: 31822
🧼 이상치 제거 완료. 정제 비율: 96.03%

### Fold 1 시작 ###
Epoch 1/15
1054/1054 [==============================] - 237s 213ms/step - loss: 2.8560 - accuracy: 0.4437 - val_loss: 4.0846 - val_accuracy: 0.1942
Epoch 2/15
1054/1054 [==============================] - 91s 84ms/step - loss: 0.8990 - accuracy: 0.7484 - val_loss: 2.4110 - val_accuracy: 0.3700
Epoch 3/15
1054/1054 [==============================] - 91s 84ms/step - loss: 0.4149 - accuracy: 0.8765 - val_loss: 1.8808 - val_accuracy: 0.4949
Epoch 4/15
1054/1054 [==============================] - 91s 84ms/step - loss: 0.2615 - accuracy: 0.9217 - val_loss: 1.6234 - val_accuracy: 0.5501
Epoch 5/15
1054/1054 [==============================] - 90s 84ms/step - loss: 0.1895 - accuracy: 0.9412 - val_loss: 1.4452 - val_accuracy: 0.6045
Epoch 6/15
1054/1054 [==============================] - 90s 84ms/step - loss: 0.1505 - accuracy: 0.9555 - val_loss: 1.3274 - val_accuracy: 0.6525
Epoch 7/15
1054/1054 [==========